In [36]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary

import matplotlib.pyplot as plt

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [41]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN().to(device)
print(model)


NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [42]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.292950  [   64/60000]
loss: 2.296004  [ 6464/60000]
loss: 2.299907  [12864/60000]
loss: 2.310343  [19264/60000]
loss: 2.309770  [25664/60000]
loss: 2.311912  [32064/60000]
loss: 2.303707  [38464/60000]
loss: 2.302188  [44864/60000]
loss: 2.303886  [51264/60000]
loss: 2.328280  [57664/60000]
Test Error: 
 Accuracy: 12.2%, Avg loss: 2.306381 

Epoch 2
-------------------------------
loss: 2.292950  [   64/60000]
loss: 2.296004  [ 6464/60000]
loss: 2.299907  [12864/60000]
loss: 2.310343  [19264/60000]
loss: 2.309770  [25664/60000]
loss: 2.311912  [32064/60000]
loss: 2.303707  [38464/60000]
loss: 2.302188  [44864/60000]
loss: 2.303886  [51264/60000]
loss: 2.328280  [57664/60000]
Test Error: 
 Accuracy: 12.2%, Avg loss: 2.306381 

Epoch 3
-------------------------------
loss: 2.292950  [   64/60000]
loss: 2.296004  [ 6464/60000]
loss: 2.299907  [12864/60000]
loss: 2.310343  [19264/60000]
loss: 2.309770  [25664/60000]
loss: 2.311912  [32064/600

In [45]:
model.

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [39]:
# 
summary(model.to_device(), input_size=(1,28,28))

RuntimeError: Tensor for argument input is on cpu but expected on mps

In [26]:
# summarize the model parameters
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param}")
    print(f"Parameter shape: {param.shape}")
    print(f"Requires gradient: {param.requires_grad}")
    print("-" * 50)


Parameter name: linear_relu_stack.0.weight
Parameter value: Parameter containing:
tensor([[ 0.0198, -0.0240, -0.0281,  ..., -0.0026,  0.0186,  0.0172],
        [-0.0209, -0.0290,  0.0044,  ...,  0.0346, -0.0354,  0.0079],
        [-0.0127, -0.0175, -0.0039,  ..., -0.0025, -0.0199,  0.0257],
        ...,
        [-0.0224,  0.0343, -0.0352,  ...,  0.0286, -0.0338,  0.0129],
        [ 0.0190,  0.0294,  0.0186,  ..., -0.0056, -0.0158,  0.0142],
        [-0.0340, -0.0249, -0.0080,  ...,  0.0003, -0.0034, -0.0322]],
       device='mps:0', requires_grad=True)
Parameter shape: torch.Size([512, 784])
Requires gradient: True
--------------------------------------------------
Parameter name: linear_relu_stack.0.bias
Parameter value: Parameter containing:
tensor([-2.8885e-02,  3.0724e-02,  2.9379e-02,  1.8816e-02, -2.6011e-02,
         3.3328e-03, -8.3296e-03,  2.3787e-02,  1.0857e-02, -1.3107e-02,
        -1.2003e-02, -2.3429e-02,  2.5075e-02, -2.0783e-02,  3.2708e-02,
        -3.0691e-03, -2.002

In [25]:
model.state_dict().items()

odict_items([('linear_relu_stack.0.weight', tensor([[ 0.0198, -0.0240, -0.0281,  ..., -0.0026,  0.0186,  0.0172],
        [-0.0209, -0.0290,  0.0044,  ...,  0.0346, -0.0354,  0.0079],
        [-0.0127, -0.0175, -0.0039,  ..., -0.0025, -0.0199,  0.0257],
        ...,
        [-0.0224,  0.0343, -0.0352,  ...,  0.0286, -0.0338,  0.0129],
        [ 0.0190,  0.0294,  0.0186,  ..., -0.0056, -0.0158,  0.0142],
        [-0.0340, -0.0249, -0.0080,  ...,  0.0003, -0.0034, -0.0322]],
       device='mps:0')), ('linear_relu_stack.0.bias', tensor([-2.8885e-02,  3.0724e-02,  2.9379e-02,  1.8816e-02, -2.6011e-02,
         3.3328e-03, -8.3296e-03,  2.3787e-02,  1.0857e-02, -1.3107e-02,
        -1.2003e-02, -2.3429e-02,  2.5075e-02, -2.0783e-02,  3.2708e-02,
        -3.0691e-03, -2.0024e-02, -3.4753e-02,  3.2016e-02, -1.2303e-02,
         3.3830e-02,  2.0743e-02,  1.3980e-02,  1.5055e-02, -2.6451e-02,
        -1.3471e-02, -6.7056e-03, -3.0503e-02,  3.6014e-02,  3.1312e-02,
         5.1512e-03,  1.1884e-

In [32]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 932362
Trainable parameters: 932362
